In [3]:
import torch
import logging
import torch.nn as nn
from tqdm import tqdm
from pathlib import Path
from torch import autograd
from sklearn import model_selection
from torch.utils.data import Dataset, Subset, DataLoader

from torcheeg.datasets import SEEDDataset
from torcheeg.datasets.constants import SEED_CHANNEL_LOCATION_DICT
import scipy
from torcheeg.transforms import Select,BandSignal,Compose,ToTensor,Lambda
from to import ToGrid
from typing import Callable, Dict, Union, List
import numpy as np
import soxr
from downsample import SetSamplingRate
from baseline import BaselineCorrection
from torcheeg.transforms import EEGTransform, Select,BandSignal,Compose,ToTensor

dataset = SEEDDataset(root_path='../../seed',
                                #   io_path='.torcheeg/datasets_1737122480267_T387E',
                        chunk_size=200,  # 1 second
                        overlap = 0,
                        offline_transform=Compose(
                                [
                                SetSamplingRate(origin_sampling_rate=200,target_sampling_rate=128),
                                BandSignal(sampling_rate=128,band_dict={'frequency_range':[8,40]})
                                ]),
                        online_transform=Compose(
                                # [ToTensor()]),
                                [ToGrid(SEED_CHANNEL_LOCATION_DICT),ToTensor()]),
                
                        label_transform=Compose([
                                Select('emotion'),
                                Lambda(lambda x: x + 1)
                                ]),
                        num_worker=8
)
print(dataset[0][0].shape) #EEG shape(1,30,128)
print(dataset[0][1])  # label (int)
print(len(dataset))

[2025-02-10 11:06:02] INFO (torcheeg/MainThread) 🔍 | Processing EEG data. Processed EEG data has been cached to .torcheeg/datasets_1739156762044_ci4j2.
[2025-02-10 11:06:02] INFO (torcheeg/MainThread) ⏳ | Monitoring the detailed processing of a record for debugging. The processing of other records will only be reported in percentage to keep it clean.
[PROCESS]: 100%|██████████| 1/1 [00:00<00:00, 1534.69it/s]

[RECORD 1_20131030.mat]: 0it [00:00, ?it/s]
[RECORD 1_20131030.mat]: 1it [00:01,  1.92s/it]
[RECORD 1_20131030.mat]: 6it [00:02,  3.92it/s]
[RECORD 1_20131030.mat]: 11it [00:02,  8.02it/s]
[RECORD 1_20131030.mat]: 16it [00:02, 12.64it/s]
[RECORD 1_20131030.mat]: 21it [00:02, 17.68it/s]
[RECORD 1_20131030.mat]: 26it [00:02, 22.80it/s]
[RECORD 1_20131030.mat]: 31it [00:02, 27.60it/s]
[RECORD 1_20131030.mat]: 36it [00:02, 31.96it/s]
[RECORD 1_20131030.mat]: 41it [00:02, 35.78it/s]
[RECORD 1_20131030.mat]: 46it [00:02, 38.85it/s]
[RECORD 1_20131030.mat]: 51it [00:02, 41.13it/s]
[RECOR

torch.Size([128, 9, 9])
2
3394


In [1]:
import torch
torch.cuda.empty_cache()


/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import logging
import torch.nn as nn
from tqdm import tqdm
from pathlib import Path
from torch import autograd
from sklearn import model_selection
from torch.utils.data import Dataset, Subset, DataLoader

from strokes import StrokePatientsMIDataset
from strokesdict import STROKEPATIENTSMI_LOCATION_DICT
import scipy
from torcheeg.transforms import Select,BandSignal,Compose
from to import ToGrid, ToTensor
from typing import Callable, Dict, Union, List
import numpy as np
import soxr
from downsample import SetSamplingRate
from baseline import BaselineCorrection
from torcheeg.transforms import EEGTransform, Select,BandSignal,Compose,ToTensor

dataset = StrokePatientsMIDataset(root_path='../../mi_swin/subdataset',
                                #   io_path='.torcheeg/datasets_1739340308797_NcxaA',
                        chunk_size=500,  # 1 second
                        overlap = 0,
                        offline_transform=Compose(
                                [BaselineCorrection(),
                                SetSamplingRate(origin_sampling_rate=500,target_sampling_rate=128),
                                BandSignal(sampling_rate=128,band_dict={'frequency_range':[8,40]})
                                ]),
                        online_transform=Compose(
                                # [ToTensor()]),
                                [ToGrid(STROKEPATIENTSMI_LOCATION_DICT),ToTensor()]),
                
                        label_transform=Select('label'),
                        num_worker=8
)
print(dataset[0][0].shape) #EEG shape(1,30,128)
print(dataset[0][1])  # label (int)
print(len(dataset))

[2025-02-12 16:56:09] INFO (torcheeg/MainThread) 🔍 | Processing EEG data. Processed EEG data has been cached to .torcheeg/datasets_1739350569768_uk8ry.
[2025-02-12 16:56:09] INFO (torcheeg/MainThread) ⏳ | Monitoring the detailed processing of a record for debugging. The processing of other records will only be reported in percentage to keep it clean.
[PROCESS]: 100%|██████████| 1/1 [00:00<00:00, 80.88it/s]

[RECORD ../../mi_swin/subdataset/sourcedata/sub-45/sub-45_task-motor-imagery_eeg.mat]: 0it [00:00, ?it/s]
[RECORD ../../mi_swin/subdataset/sourcedata/sub-45/sub-45_task-motor-imagery_eeg.mat]: 1it [00:00,  4.46it/s]
[RECORD ../../mi_swin/subdataset/sourcedata/sub-45/sub-45_task-motor-imagery_eeg.mat]: 2it [00:00,  5.86it/s]
[RECORD ../../mi_swin/subdataset/sourcedata/sub-45/sub-45_task-motor-imagery_eeg.mat]: 12it [00:00, 36.57it/s]
[RECORD ../../mi_swin/subdataset/sourcedata/sub-45/sub-45_task-motor-imagery_eeg.mat]: 22it [00:00, 56.46it/s]
[RECORD ../../mi_swin/subdataset/sourceda

torch.Size([128, 9, 9, 1])
0
160


In [3]:
import os
import shutil

def delete_folder_if_exists(target_folder_name):
    # 获取父文件夹中的所有内容
    parent_folder = os.getcwd()
    for folder_name in os.listdir(parent_folder):
        folder_path = os.path.join(parent_folder, folder_name)

        # 检查是否是文件夹并且名称是否匹配
        if os.path.isdir(folder_path) and folder_name == target_folder_name:
            try:
                # 删除目标文件夹
                shutil.rmtree(folder_path)
                print(f"已删除文件夹: {folder_path}")
            except Exception as e:
                print(f"删除文件夹 {folder_path} 时出错: {e}")

In [4]:
from eegswintransformer import SwinTransformer

HYPERPARAMETERS = {
    "seed": 42,
    "batch_size": 4,
    "lr": 1e-4,
    "weight_decay": 1e-5,
    "num_epochs": 20,
}
from torcheeg.model_selection import KFoldPerSubjectGroupbyTrial
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torch.utils.data import DataLoader
from classifier import ClassifierTrainer
k_fold = KFoldPerSubjectGroupbyTrial(
    n_splits=4,
    shuffle=True,
    random_state=42)

training_metrics = []
test_metrics = []

for i, (training_dataset, test_dataset) in enumerate(k_fold.split(dataset)):
    delete_folder_if_exists(target_folder_name='lightning_logs')
    model = SwinTransformer(patch_size=(32,3,3),depths=(2, 2, 2, 2), num_heads=(1, 1, 1, 1))
    trainer = ClassifierTrainer(model=model,
                                num_classes=2,
                                lr=HYPERPARAMETERS['lr'],
                                weight_decay=HYPERPARAMETERS['weight_decay'],
                                metrics=["accuracy"],
                                accelerator="gpu")
    
    training_loader = DataLoader(training_dataset,
                             batch_size=HYPERPARAMETERS['batch_size'],
                             shuffle=True)
    test_loader = DataLoader(test_dataset,
                             batch_size=HYPERPARAMETERS['batch_size'],
                             shuffle=False)
    # 提前停止回调
    early_stopping_callback = EarlyStopping(
        monitor='train_loss',
        patience=10, 
        mode='min',
        verbose=True
    )
    
    trainer.fit(training_loader,
                test_loader,
                max_epochs=HYPERPARAMETERS['num_epochs'],
                callbacks=[early_stopping_callback],
                # enable_progress_bar=True,
                enable_model_summary=False,
                limit_val_batches=0.0)
    # training_result = trainer.test(training_loader,
    #                                enable_progress_bar=True,
    #                                enable_model_summary=True)[0]
    test_result = trainer.test(test_loader,
                               enable_progress_bar=True,
                               enable_model_summary=True)[0]
    # training_metrics.append(training_result["test_accuracy"])
    test_metrics.append(test_result["test_accuracy"])
     
print({
    # "training_metric_avg": np.mean(training_metrics),
    # "training_metric_std": np.std(training_metrics),
    "test_metric_avg": np.mean(test_metrics),
    "test_metric_std": np.std(test_metrics)
})

[2025-02-12 16:56:21] INFO (torcheeg/MainThread) 📊 | Create the split of train and test set.
[2025-02-12 16:56:21] INFO (torcheeg/MainThread) 😊 | Please set split_path to .torcheeg/model_selection_1739350581083_W1YOf for the next run, if you want to use the same setting for the experiment.
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarni

Epoch 0: 100%|██████████| 30/30 [00:05<00:00,  5.12it/s, loss=6.65, train_loss=1.400, train_accuracy=0.250]  

Metric train_loss improved. New best score: 1.401


Epoch 0: 100%|██████████| 30/30 [00:05<00:00,  5.12it/s, loss=6.65, train_loss=1.400, train_accuracy=0.250]

/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_accuracy', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
[2025-02-12 16:56:41] INFO (torcheeg/MainThread) 
[Train] train_loss: 8.975 train_accuracy: 0.558 



...

Monitored metric train_loss did not improve in the last 10 records. Best score: 0.000. Signaling Trainer to stop.


Epoch 17: 100%|██████████| 30/30 [00:05<00:00,  5.96it/s, loss=0.000117, train_loss=5.39e-6, train_accuracy=1.000]

[2025-02-12 17:04:04] INFO (torcheeg/MainThread) 
[Train] train_loss: 0.000 train_accuracy: 1.000 



Epoch 17: 100%|██████████| 30/30 [00:05<00:00,  5.96it/s, loss=0.000117, train_loss=5.39e-6, train_accuracy=1.000]


: 

In [8]:
for i, score in enumerate(test_metrics):
    print(i,score)

0 0.8999999761581421
1 0.824999988079071
2 0.7749999761581421
3 0.824999988079071
4 0.5249999761581421
5 0.625
6 0.550000011920929
7 0.6499999761581421
